In [1]:
!pip install wandb pathling pyspark nibabel minio requests boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 26.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 8.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.9/135.9 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 kB 61.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 39.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 30.

In [2]:
import wandb
import os
from pathling.coding import Coding
from pyspark.sql import functions
from pathling.udfs import subsumes
from utils import configure_notebook_session, get_spark_session, get_pathling_context, \
load_resources, extract_patient_id, extract_subject_id, save_artifact

In [4]:
# Configure some constants for the demo

PROJECT_NAME = 'diabetes-vaccines-notebook'
NOTEBOOK_NAME = 'prepare-data.ipynb'
ARTIFACT_NAME = 'training-data'
RESOURCES = ['Patient', 'Immunization', 'Condition']
START_BIRTH_DATE = '1920-01-01'
END_BIRTH_DATE = '2020-01-01'
IMMUNIZATION_CODE = '08' # Hepatitis B
CONDITION_CODE = '73211009' # Diabetes

# Configure notebook and login to wandb
configure_notebook_session(NOTEBOOK_NAME)
wandb.login()

wandb: You can find your API key in your browser here: http://wandb:8080/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: ERROR Host must be a url in the form https://some.address.com, received http://wandb:8080
wandb: Currently logged in as: louismullie (louism). Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [ ]:
# Configure Spark, Pathling, and load resources
run = wandb.init(project=PROJECT_NAME, job_type="data_convert")
spark = get_spark_session()
pc = get_pathling_context(spark)
resource_data = load_resources(pc, RESOURCES)

In [5]:
###### 1. FILTERING INITIAL PATIENTS ######

# Filtering patients based on birthdate
patients = resource_data['Patient']
filtered_patients = patients.filter(
    (patients.birthDate < END_BIRTH_DATE) & 
    (patients.birthDate > START_BIRTH_DATE))

filtered_patients.select('gender', 'birthDate').show()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


+------+----------+
|gender| birthDate|
+------+----------+
|female|2019-07-16|
|female|1974-01-29|
|  male|1945-07-09|
|  male|2016-06-02|
|female|1996-05-28|
|female|2017-11-04|
|  male|1955-08-27|
|female|1942-10-19|
|  male|1997-04-10|
|  male|1973-03-30|
|female|1966-03-08|
|  male|1945-07-09|
|  male|1955-08-27|
|  male|1951-07-26|
|  male|1953-03-09|
|  male|1987-06-06|
|  male|1971-01-09|
|female|2005-04-23|
|female|1936-10-27|
|female|1992-10-01|
+------+----------+
only showing top 20 rows



In [6]:
###### 2. SELECTING HEP B IMMUNIZATIONS ######

# Joining filtered patients with immunizations
immunizations = resource_data['Immunization']
patients_immunizations = filtered_patients.join(
    immunizations.select('patient', 'vaccineCode'),
    filtered_patients.id == extract_patient_id(immunizations),
    'left_outer'
)

# Selecting patient id and their vaccine coding
patients_immunizations = patients_immunizations.select(
    filtered_patients.id,
    functions.explode(
        patients_immunizations.vaccineCode.coding.getField('code')
    ).alias('code')
)

# Checking patients who received the hepatitis B vaccine
patients_immunizations = patients_immunizations.withColumn(
    'is_vaccinated',
    patients_immunizations.code == IMMUNIZATION_CODE
)

patients_immunizations.show()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


+--------------------+----+-------------+
|                  id|code|is_vaccinated|
+--------------------+----+-------------+
|d9b90399-0504-4f1...|  08|         true|
|e0f57407-86ed-405...| 140|        false|
|e9adac47-eb98-4fc...|  08|         true|
|55c11701-518c-4ef...|  08|         true|
|d9b90399-0504-4f1...|  08|         true|
|55c11701-518c-4ef...|  08|         true|
|e0f57407-86ed-405...| 140|        false|
|d9b90399-0504-4f1...|  49|        false|
|d9b90399-0504-4f1...| 119|        false|
|d9b90399-0504-4f1...|  10|        false|
|d9b90399-0504-4f1...|  20|        false|
|55c11701-518c-4ef...|  49|        false|
|d9b90399-0504-4f1...| 133|        false|
|55c11701-518c-4ef...| 119|        false|
|55c11701-518c-4ef...|  10|        false|
|55c11701-518c-4ef...|  20|        false|
|55c11701-518c-4ef...| 133|        false|
|d9b90399-0504-4f1...|  49|        false|
|d9b90399-0504-4f1...| 119|        false|
|d9b90399-0504-4f1...|  10|        false|
+--------------------+----+-------

In [7]:
###### 3. SELECTING DIABETIC PATIENTS ######

# Joining filtered patients with conditions
conditions = resource_data['Condition']
patients_conditions = filtered_patients.join(
    conditions.select('subject', 'code'),
    filtered_patients.id == extract_subject_id(conditions),
    'left_outer'
)

# Selecting patient id and their condition coding
patients_conditions = patients_conditions.select(
    filtered_patients.id,
    functions.explode_outer(conditions.code.getField('coding')) \
    .alias('codings')
)

# Checking patients who have diabetes
condition_coding = Coding(system='http://snomed.info/sct', code=CONDITION_CODE)
patients_conditions = patients_conditions.withColumn(
    'has_diabetes',
    subsumes(condition_coding, patients_conditions.codings)                                 
)

patients_conditions.show()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


+--------------------+--------------------+------------+
|                  id|             codings|has_diabetes|
+--------------------+--------------------+------------+
|e9adac47-eb98-4fc...|                null|        null|
|c60ac337-2ebe-48f...|{null, http://sno...|       false|
|c60ac337-2ebe-48f...|{null, http://sno...|       false|
|c60ac337-2ebe-48f...|{null, http://sno...|       false|
|c60ac337-2ebe-48f...|{null, http://sno...|       false|
|c60ac337-2ebe-48f...|{null, http://sno...|       false|
|c60ac337-2ebe-48f...|{null, http://sno...|       false|
|c60ac337-2ebe-48f...|{null, http://sno...|       false|
|c60ac337-2ebe-48f...|{null, http://sno...|       false|
|c60ac337-2ebe-48f...|{null, http://sno...|       false|
|cc31e8b9-9d75-4f7...|{null, http://sno...|       false|
|cc31e8b9-9d75-4f7...|{null, http://sno...|       false|
|cc31e8b9-9d75-4f7...|{null, http://sno...|       false|
|cc31e8b9-9d75-4f7...|{null, http://sno...|       false|
|cc31e8b9-9d75-4f7...|{null, ht

In [8]:
###### 4. JOINING THE DIFFERENT DATA SOURCES ######

# Joining vaccine and diabetes dataframes to identify unvaccinated high risk patients
df_a = patients_immunizations.withColumn('id_a', patients_immunizations.id)
df_b = patients_conditions.withColumn('id_b', patients_conditions.id)
two_by_two = df_a.join(df_b, functions.col('id_a') == functions.col('id_b'), 'left_outer')
 
# Aggregating data to get a 2x2 table of diabetes (Y/N) and vaccination (Y/N)
aggregate = two_by_two.groupBy(
    patients_immunizations.is_vaccinated, 
    patients_conditions.has_diabetes
).agg(
    functions.countDistinct(patients_immunizations.id)
)

# Display the results
aggregate.show()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


+-------------+------------+---------+
|is_vaccinated|has_diabetes|count(id)|
+-------------+------------+---------+
|         true|       false|       18|
|         true|        null|        2|
|        false|       false|      114|
|        false|        true|        8|
|        false|        null|        1|
+-------------+------------+---------+



In [9]:
# Optionally, log it as a wandb Table for viewing
table = wandb.Table(dataframe=aggregate.toPandas())

run.log({"hep_b_vaccination_in_diabetics": table})

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [10]:
# Write the results to Minio in Parquet format using Spark
save_artifact(aggregate, PROJECT_NAME, ARTIFACT_NAME, run)

wandb: Generating checksum for up to 10000 objects with prefix "datasets/diabetes-vaccines-notebook/training-data.parquet"... Done. 0.0s
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [11]:
run.finish()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
